# Project: school and major select system

## Porject purpose
In today's complex educational landscape, choosing the right school an dmajor can be a daunting task for sutdents and their families. To simplify this process and empower individuals with data-driven insights, we propose the creation of an Education Decision Support System. This system will leverage a comprehensive dataset of educational information to provide valuable guidance and visualization tools for informed decision-making. The purpose of the ducational decision support system is to assist students and their families in making well-informed choices regarding their education by harnessing the power of data analytics and visualization. The platform aims to achieve the following key objectives: 
<br> &emsp;&emsp;a. search a list or visualization chart of university based on the user's perspective 
<br> &emsp;&emsp;b. search a list or visualization chart of major based on the user's perspective
## Data resource
<br> 1. U.S. Bureau of Labor Statistics (gov)
<br> 2. National Center for Education Statistics (gov)
<br> 3. Post-Secondary Employment Outcomes (PSEO) - Census (gov)
## Example
<br> Example 1: 
     <br> &emsp;&emsp;The user comes from a poor family from SD. They want to find a major that can be complete in 4 years and find a job in 1 year. 
<br> Step: 
     <br> &emsp;&emsp;1. change or select the mod to "find a major" 
     <br> &emsp;&emsp;2. change the filter options. location to SD, 
     <br> &emsp;&emsp;3. change sort by: % of instate employed (y1_grads_emp_instate / (y1_grads_emp + y1_grads_nme ))
<br> output :
     <br> &emsp;&emsp;col1: major list (based on cipcode) in SD 
     <br> &emsp;&emsp;col2: percentage of employed 
     <br> &emsp;&emsp;col3: the top 3 percentage of industry type that graduates are employed in 
<br>
<br> Example 2: 
<br> &emsp;&emsp;The student wants to work in school system and get a better development potential. He doesn't consider pursuing a PhD. 
<br> Step: 
    <br>&emsp;&emsp; 1. use function to search cip code  which include "school" or "university" in description
    <br>&emsp;&emsp; 2. change or select the mod to "find a major" 
    <br>&emsp;&emsp; 3. add filter cip-code: add / select all previous search result
    <br>&emsp;&emsp; 4. add filter degree level: select associate, certificate, bachelor, master. 
    <br>&emsp;&emsp; 5. sort by mean_salary 
<br> Output: 
    <br>&emsp;&emsp; major list (based on cipcode) 
    <br>&emsp;&emsp; top 5 related SOC (job title) 
    <br>&emsp;&emsp; top 3 related industry (include school but not limited in school) 
    <br>&emsp;&emsp; 10_year employed rate

In [1]:
import pandas as pd
import matplotlib as plt
import numpy as mp
import seaborn as sb
import tkinter as tk
import gc

In [3]:
CIPtoSOC = pd.read_excel("CIP2020_SOC2018_Crosswalk.xlsx",sheet_name = "SOC-CIP")
label_cip_level = pd.read_csv("label_cip_level.csv")
label_cipcode = pd.read_csv("label_cipcode.csv")
label_degree_level = pd.read_csv("label_degree_level.csv")
label_fipsnum = pd.read_csv("label_fipsnum.csv")
label_geo_level = pd.read_csv("label_geo_level.csv")
label_geography_division = pd.read_csv("label_geography_division.csv")
label_ind_level = pd.read_csv("label_ind_level.csv")
label_industry = pd.read_csv("label_industry.csv")
label_institution = pd.read_csv("label_institution.csv")
pseof_all = pd.read_csv("pseof_all.csv")
pseoe_all = pd.read_csv("pseoe_all.csv")
national_M2022 = pd.read_excel("national_M2022_dl.xlsx")

C:\Users\py198\AppData\Local\Temp\ipykernel_16736\1005307885.py:11: DtypeWarning: Columns (2,4,11) have mixed types. Specify dtype option on import or set low_memory=False.
  pseof_all = pd.read_csv("pseof_all.csv")


## pseof_all

In [36]:
pseof_all = pd.read_csv("pseof_all.csv",dtype = 'str')
pseoe_all = pd.read_csv("pseoe_all.csv", dtype = 'str')
pseof_all = pseof_all.merge(pseoe_all, on = ['institution','agg_level_pseo','inst_level','degree_level','cip_level',
                                       'grad_cohort','grad_cohort_years','geo_level','geography','cipcode'], how = 'left')
pseof_all.drop(['status_y1_earnings', 'status_y1_grads_earn', 'status_y5_earnings',
       'status_y5_grads_earn', 'status_y10_earnings', 'status_y10_grads_earn',
       'status_y1_ipeds_count', 'status_y5_ipeds_count',
       'status_y10_ipeds_count','status_y1_grads_emp',
       'status_y1_grads_emp_instate', 'status_y5_grads_emp',
       'status_y5_grads_emp_instate', 'status_y10_grads_emp',
       'status_y10_grads_emp_instate', 'status_y1_grads_nme',
       'status_y5_grads_nme', 'status_y10_grads_nme','ind_level_y','industry_y'],axis = 1, inplace = True)
pseof_all['ind_level'] = pseof_all['ind_level_x']
pseof_all['industry'] = pseof_all['industry_x']
del pseof_all['industry_x']
del pseof_all['ind_level_x']
pseof_all['institution'] = pseof_all['institution'].str.lstrip('0')

label_institution = pd.read_csv("label_institution.csv").astype(str)
merged_df = pseof_all.merge(label_institution, on = "institution", how = "left")
merged_df['institution'] = merged_df['label']
del merged_df['label']
del merged_df['inst_level_x']
del merged_df['inst_level_y']
del merged_df['statefips']
del pseof_all
del label_institution


label_degree_level = pd.read_csv("label_degree_level.csv", dtype  = 'str')
merged_df = merged_df.merge(label_degree_level, on = "degree_level", how = "left")
merged_df['degree_level'] = merged_df['label']
del merged_df['label']
del label_degree_level

label_cipcode = pd.read_csv("label_cipcode.csv", dtype = 'str')
merged_df = merged_df.merge(label_cipcode, on = "cipcode", how = "left")
del merged_df['cip_level_y']
merged_df['cip_level'] = merged_df['cip_level_x']
del merged_df['cip_level_x']
merged_df['cip_name'] = merged_df['label']
del merged_df['label']
merged_df['cip_definition'] = merged_df['CIPDefinition']
del merged_df['CIPDefinition']
del label_cipcode

label_industry = pd.read_csv("label_industry.csv", dtype = 'str')
merged_df = merged_df.merge(label_industry, on = "industry", how = "left")

merged_df['industry_name'] = merged_df['label']
merged_df['ind_level'] = merged_df['ind_level_x']
del merged_df['ind_level_x']
del merged_df['ind_level_y']
del merged_df['label']
del label_industry

label_geography_division = pd.read_csv("label_geography_division.csv", dtype = 'str')
merged_df = merged_df.merge(label_geography_division, on = "geography", how = "left")
merged_df.tail()
merged_df['geography'] = merged_df['label']
merged_df['geo_level'] = merged_df['geo_level_x']
del merged_df['label']
del merged_df['geo_level_x']
del merged_df['geo_level_y']
del label_geography_division
del merged_df['agg_level_pseo']
del merged_df['geography']

In [44]:
merged_df.to_csv('merged_df.csv')

In [49]:
import pandas as pd
all_data_M_2022 = pd.read_excel("all_data_M_2022.xlsx", dtype = 'str')
CIP_SOC_Crosswalk = pd.read_excel("CIP2020_SOC2018_Crosswalk.xlsx", sheet_name = "CIP-SOC", dtype = 'str')
CIP_SOC_Crosswalk['CIPFamily'] = CIP_SOC_Crosswalk['CIP2020Code'].str[:2]
CIP_SOC_Crosswalk['CIP2020Code'] = CIP_SOC_Crosswalk['CIP2020Code'].str.replace('.','-')
all_data_M_2022['SOC2018Code'] = all_data_M_2022['OCC_CODE']
del all_data_M_2022['OCC_CODE']


C:\Users\py198\AppData\Local\Temp\ipykernel_3448\691805225.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  CIP_SOC_Crosswalk['CIP2020Code'] = CIP_SOC_Crosswalk['CIP2020Code'].str.replace('.','-')


In [50]:
all_data_M_2022.head()

,AREA,AREA_TITLE,AREA_TYPE,PRIM_STATE,NAICS,NAICS_TITLE,I_GROUP,OWN_CODE,OCC_TITLE,O_GROUP,...,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY,SOC2018Code
0,99,U.S.,1,US,000000,Cross-industry,cross-industry,1235,All Occupations,total,...,35.32,53.03,27340,33330,46310,73460,110290,NaN,NaN,00-0000
1,99,U.S.,1,US,000000,Cross-industry,cross-industry,1235,Management Occupations,major,...,78.71,106.03,50290,75350,107360,163710,220550,NaN,NaN,11-0000
2,99,U.S.,1,US,000000,Cross-industry,cross-industry,1235,Top Executives,minor,...,76.96,#,43440,62520,99890,160070,#,NaN,NaN,11-1000
3,99,U.S.,1,US,000000,Cross-industry,cross-industry,1235,Chief Executives,broad,...,#,#,74920,122480,189520,#,#,NaN,NaN,11-1010
4,99,U.S.,1,US,000000,Cross-industry,cross-industry,1235,Chief Executives,detailed,...,#,#,74920,122480,189520,#,#,NaN,NaN,11-1011


In [52]:
CIP_SOC_Crosswalk.head()

,CIP2020Code,CIP2020Title,SOC2018Code,SOC2018Title,CIPFamily
0,01-0000,"Agriculture, General.",19-1011,Animal Scientists,01
1,01-0000,"Agriculture, General.",19-1012,Food Scientists and Technologists,01
2,01-0000,"Agriculture, General.",19-1013,Soil and Plant Scientists,01
3,01-0000,"Agriculture, General.",19-4012,Agricultural Technicians,01
4,01-0000,"Agriculture, General.",25-1041,"Agricultural Sciences Teachers, Postsecondary",01


In [96]:
pseo = pd.read_csv("merged_df.csv")
pseo = pseo.drop(pseo.columns[0],axis = 1)

C:\Users\py198\AppData\Local\Temp\ipykernel_3448\3367987446.py:1: DtypeWarning: Columns (30,34) have mixed types. Specify dtype option on import or set low_memory=False.
  pseo = pd.read_csv("merged_df.csv")


In [59]:
all_data_M_2022.columns

Index(['AREA', 'AREA_TITLE', 'AREA_TYPE', 'PRIM_STATE', 'NAICS', 'NAICS_TITLE',
       'I_GROUP', 'OWN_CODE', 'OCC_TITLE', 'O_GROUP', 'TOT_EMP', 'EMP_PRSE',
       'JOBS_1000', 'LOC_QUOTIENT', 'PCT_TOTAL', 'PCT_RPT', 'H_MEAN', 'A_MEAN',
       'MEAN_PRSE', 'H_PCT10', 'H_PCT25', 'H_MEDIAN', 'H_PCT75', 'H_PCT90',
       'A_PCT10', 'A_PCT25', 'A_MEDIAN', 'A_PCT75', 'A_PCT90', 'ANNUAL',
       'HOURLY', 'SOC2018Code'],
      dtype='object')

In [92]:
pseo.columns

Index(['institution', 'degree_level', 'cipcode', 'grad_cohort',
       'grad_cohort_years', 'y1_grads_emp', 'y1_grads_emp_instate',
       'y5_grads_emp', 'y5_grads_emp_instate', 'y10_grads_emp',
       'y10_grads_emp_instate', 'y1_grads_nme', 'y5_grads_nme',
       'y10_grads_nme', 'y1_p25_earnings', 'y1_p50_earnings',
       'y1_p75_earnings', 'y1_grads_earn', 'y5_p25_earnings',
       'y5_p50_earnings', 'y5_p75_earnings', 'y5_grads_earn',
       'y10_p25_earnings', 'y10_p50_earnings', 'y10_p75_earnings',
       'y10_grads_earn', 'y1_ipeds_count', 'y5_ipeds_count', 'y10_ipeds_count',
       'industry', 'city', 'institution_state', 'CIPFamily', 'cip_level',
       'cip_name', 'cip_definition', 'industry_name', 'ind_level',
       'geo_level'],
      dtype='object')

In [108]:
import tkinter as tk
from tkinter import ttk

institution_all = pseo['institution'].unique().tolist()
degree_level_all = pseo['degree_level'].unique().tolist()
cip_name_all = pseo['cip_name'].unique().tolist()
state_all = pseo['institution_state'].unique().tolist()
industry_all = pseo['industry_name'].unique().tolist()

